In [2]:
import pandas as pd
import numpy as np
import requests
import json
import csv

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
# pd.set_option('display.max_colwidth', None)

# Loading, looping through, flattening, and formatting the Json file from the API Querying Notebook
Here I am defining a function to pull the data I want from the queried json files.

The format is divided into teams, which is why we loop through each team color (there are only blue and orange for Rocket League). I then pull player info and loop through each player for their stats. Specifically player_id and player_name sometimes had nulls (as some players use weird characters for names that do not translate well onto ballchasing.com and result in nulls), so setting a placeholder for the few instances of that was necessary. 

In [4]:
def load_and_flatten_data(filename):
    flattened_data = []
    with open(filename, 'r') as file:
        for line in file:
            replays = json.loads(line)
            for replay in replays:
                replay_id = replay.get('id')
                for team_color in ['blue', 'orange']:
                    team = replay.get(team_color, {})
                    players = team.get('players', [])
                    for player in players:
                        player_stats = player.get('stats', {})
                        flattened_stats = {
                            'replay_id': replay_id,
                            'player_id': player['id'].get('id', 'Unknown ID'),
                            'player_name': player.get('name', 'Unknown Name'),
                            'team_color': team_color,
                            **{k: v for section in player_stats.values() for k, v in section.items()}
                        }
                        flattened_data.append(flattened_stats)
    return pd.DataFrame(flattened_data)


filename = 'C:/Users/nickh/Desktop/BrainStation Course/Capstone Project RL/AllData35000to50000NoProcessing.json'
df_test = load_and_flatten_data(filename)

In [5]:
df_test.shape

(89998, 88)

# EDA/Feature Engineering
The EDA for this dataset did not require much, since the data coming from ballchasing.com was already quite robust. Most of my time for this part was spent on feature engineering and utilizing my knowledge of the game to make decisions on what to include and how to handle the data

### Important EDA Finding
Below is a list of all the stats that are available for each player in each game. There are many stats that have two formats, either percentage or time, of which I picked percentage in every case. This was due to the fact that, in Rocket League, quite a few games end in a forfeit either due to rage quitting or one team is simply up to much and the other team forfeits. In a game that was forfeit, a player may have done very well but still have a lower score on a stat that only showed total time. The stats that use percentage would not be affected by a game that ran for less than the full 5 minutes, or for that matter, over 5 minutes in the event of an overtime (if scores are tied at the end of the game).

In [6]:
df_test.columns

Index(['replay_id', 'player_id', 'player_name', 'team_color', 'shots',
       'shots_against', 'goals', 'goals_against', 'saves', 'assists', 'score',
       'mvp', 'shooting_percentage', 'bpm', 'bcpm', 'avg_amount',
       'amount_collected', 'amount_stolen', 'amount_collected_big',
       'amount_stolen_big', 'amount_collected_small', 'amount_stolen_small',
       'count_collected_big', 'count_stolen_big', 'count_collected_small',
       'count_stolen_small', 'amount_overfill', 'amount_overfill_stolen',
       'amount_used_while_supersonic', 'time_zero_boost', 'percent_zero_boost',
       'time_full_boost', 'percent_full_boost', 'time_boost_0_25',
       'time_boost_25_50', 'time_boost_50_75', 'time_boost_75_100',
       'percent_boost_0_25', 'percent_boost_25_50', 'percent_boost_50_75',
       'percent_boost_75_100', 'avg_speed', 'total_distance',
       'time_supersonic_speed', 'time_boost_speed', 'time_slow_speed',
       'time_ground', 'time_low_air', 'time_high_air', 'time_powers

### Creating the y Variable we want to Predict
Since there was no column in the dataset that showed a win or loss, I created a new column that compared the 'goals_against' column for each team to find out which team won, and then applied that value accordingly across all 6 players (with 1 row being one player's stats/data) in the new column.

In [7]:
def assign_win_loss(group):
    goals_against_blue = group[group['team_color'] == 'blue']['goals_against'].iloc[0]
    goals_against_orange = group[group['team_color'] == 'orange']['goals_against'].iloc[0]
    
    if goals_against_blue < goals_against_orange:
        group['win'] = group['team_color'].apply(lambda x: 1 if x == 'blue' else 0)
    else:
        group['win'] = group['team_color'].apply(lambda x: 1 if x == 'orange' else 0)
    
    return group


In [8]:
# applying the win/loss function to the dataset
df_test_wins = df_test.groupby('replay_id').apply(assign_win_loss).reset_index(drop=True)

In [34]:
# testing for nulls
df_test_wins.isna().sum()

replay_id                              0
player_id                              0
player_name                            0
team_color                             0
bpm                                    0
bcpm                                   0
amount_collected                       0
amount_stolen                          0
amount_collected_big                   0
amount_stolen_big                      0
amount_collected_small                 0
amount_stolen_small                    0
amount_overfill                        0
amount_overfill_stolen                 0
amount_used_while_supersonic           0
percent_zero_boost                     0
percent_full_boost                     0
percent_boost_0_25                     0
percent_boost_25_50                    0
percent_boost_50_75                    0
percent_boost_75_100                   0
avg_speed                              0
time_supersonic_speed                  0
time_boost_speed                       0
time_slow_speed 

### Important EDA Finding
As you can see in the head of the data below, every 6 rows is one game of Rocket League, with each row containing the stats for one player in the game. Teams are denoted by the 'team_color' column, of which there are only two values: blue (which is always first, as that is how the data comes from the API) and orange.

Due to this, before inputting the data into any model, I had to (further down in the notebook) flatten each team's data into a 2d array that contained all the stats for each player on that team, and then feed the two arrays into the models for training.

In [10]:
df_test_wins.head(6)

,replay_id,player_id,player_name,team_color,shots,shots_against,goals,goals_against,saves,assists,score,mvp,shooting_percentage,bpm,bcpm,avg_amount,amount_collected,amount_stolen,amount_collected_big,amount_stolen_big,amount_collected_small,amount_stolen_small,count_collected_big,count_stolen_big,count_collected_small,count_stolen_small,amount_overfill,amount_overfill_stolen,amount_used_while_supersonic,time_zero_boost,percent_zero_boost,time_full_boost,percent_full_boost,time_boost_0_25,time_boost_25_50,time_boost_50_75,time_boost_75_100,percent_boost_0_25,percent_boost_25_50,percent_boost_50_75,percent_boost_75_100,avg_speed,total_distance,time_supersonic_speed,time_boost_speed,time_slow_speed,time_ground,time_low_air,time_high_air,time_powerslide,count_powerslide,avg_powerslide_duration,avg_speed_percentage,percent_slow_speed,percent_boost_speed,percent_supersonic_speed,percent_ground,percent_low_air,percent_high_air,avg_distance_to_ball,avg_distance_to_ball_possession,avg_distance_to_ball_no_possession,avg_distance_to_mates,time_defensive_third,time_neutral_third,time_offensive_third,time_defensive_half,time_offensive_half,time_behind_ball,time_infront_ball,time_most_back,time_most_forward,time_closest_to_ball,time_farthest_from_ball,percent_defensive_third,percent_offensive_third,percent_neutral_third,percent_defensive_half,percent_offensive_half,percent_behind_ball,percent_infront_ball,percent_most_back,percent_most_forward,percent_closest_to_ball,percent_farthest_from_ball,inflicted,taken,goals_against_while_last_defender,win
0,00014aaf-22c9-4712-a4ec-de28a60dbc6d,76561198213154948,David,blue,5,6,2,1,3,0,622,True,40.0,279,269.47995,45.09,1837,598,857,231,980,367,11,3,85,32,254,73,119,24.59,6.012078,17.09,4.178382,135.78,149.69,64.94,53.67,33.602257,37.044643,16.071074,13.282022,1516,604291,42.61,186.11,185.91,216.01,176.85,21.77,5.43,47,0.12,65.913040,44.837566,44.885800,10.276632,52.097054,42.652490,5.250464,3030,3102,2929,3529.0,175.87,120.31,118.45,234.46,180.17,293.78,120.86,146.3,134.0,115.8,170.3,42.416130,28.567638,29.016232,56.546800,43.453197,70.851820,29.148176,35.769295,32.762035,28.312265,41.637123,0,2,NaN,1
1,00014aaf-22c9-4712-a4ec-de28a60dbc6d,d58a79bcdbbd4a86bc5c964037f53a25,EnigmaaaaaaaaaRL,blue,4,6,0,1,2,2,422,False,0.0,476,516.80884,55.80,3523,1121,2581,786,942,335,34,10,92,35,894,240,617,43.46,10.625657,64.25,15.708662,105.16,92.72,84.60,130.30,25.476038,22.462328,20.495178,31.566450,1654,667225,81.22,181.37,155.18,242.42,152.56,22.79,15.57,109,0.14,71.913040,37.144840,43.413840,19.441320,58.027145,36.517700,5.455155,2768,2758,2774,3536.0,164.41,157.21,96.15,240.13,177.64,281.01,136.76,134.6,141.2,152.5,134.7,39.354187,23.015055,37.630753,57.478992,42.521004,67.264280,32.735710,32.908733,34.522380,37.285152,32.933178,1,1,1.0,1
2,00014aaf-22c9-4712-a4ec-de28a60dbc6d,76561198884555998,AJ,blue,2,6,0,1,0,0,196,False,0.0,387,389.03693,56.89,2652,567,2012,395,640,172,27,5,58,17,383,108,216,62.55,15.293024,54.74,13.383535,134.41,73.12,64.08,138.53,32.771736,17.828060,15.623934,33.776270,1513,602688,51.76,163.74,198.46,240.16,150.64,23.16,9.17,86,0.11,65.782610,47.941826,39.554546,12.503623,58.015270,36.389990,5.594744,2553,2465,2623,3405.0,157.69,156.32,99.94,240.72,173.24,274.01,139.95,137.2,142.3,150.3,112.5,38.093970,24.143011,37.763016,58.150543,41.849453,66.192380,33.807613,33.544410,34.791325,36.747270,27.505440,0,2,NaN,1
3,00014aaf-22c9-4712-a4ec-de28a60dbc6d,76561198089804347,noah,orange,1,11,1,2,3,0,412,False,100.0,364,355.44363,39.08,2423,363,1479,100,944,263,17,1,80,22,133,0,261,128.48,31.412434,27.27,6.667318,214.06,57.01,66.72,71.00,52.364292,13.946035,16.321339,17.368330,1491,598832,41.59,184.38,191.79,221.73,171.09,24.94,9.99,88,0.11,64.826090,45.909138,44.135390,9.955478,53.075928,40.954136,5.969935,3022,2926,3128,3717.0,219.26,131.35,67.15,295.04,122.72,281.50,136.27,140.9,130.6,134.5,147.3,52.484680,16.073822,31.441500,70.624280,29.375717,67.381570,32.618423,34.449030,31.930761,32.884280,3

In [70]:
# df_test_wins.to_csv('C:/Users/nickh/Desktop/BrainStation Course/Capstone Project RL/FullData0to35000.csv')

## More EDA

First, I dropped many columns that, using my game knowledge, I knew would not be as highly correlated to wins as the others.

I then convert all the numeric columns to float64, as some came as num64 and some as float64, so just for consistancy and modeling purposes. As well as changing the 'win' column to boolean.

In [132]:
# removing obviously unneccessary columns first
df_dropped1 = df_test_wins.drop(columns=[
    'shots', 
    'shots_against', 
    'goals', 
    'goals_against',
    'saves',
    'assists',
    'score',
    'mvp',
    'shooting_percentage',
    'goals_against_while_last_defender',
    'avg_amount',
    'count_collected_big',
    'count_stolen_big',
    'count_collected_small',
    'count_stolen_small',
    'time_zero_boost',
    'time_full_boost',
    'time_boost_0_25',
    'time_boost_25_50',
    'time_boost_50_75',
    'time_boost_75_100',
    'total_distance',
    'percent_defensive_half',
    'percent_offensive_half',
    'inflicted',
    'taken'
])

In [12]:
df_dropped1.shape

(89998, 63)

In [40]:
df_dropped1.columns

Index(['replay_id', 'player_id', 'player_name', 'team_color', 'bpm', 'bcpm',
       'amount_collected', 'amount_stolen', 'amount_collected_big',
       'amount_stolen_big', 'amount_collected_small', 'amount_stolen_small',
       'amount_overfill', 'amount_overfill_stolen',
       'amount_used_while_supersonic', 'percent_zero_boost',
       'percent_full_boost', 'percent_boost_0_25', 'percent_boost_25_50',
       'percent_boost_50_75', 'percent_boost_75_100', 'avg_speed',
       'time_supersonic_speed', 'time_boost_speed', 'time_slow_speed',
       'time_ground', 'time_low_air', 'time_high_air', 'time_powerslide',
       'count_powerslide', 'avg_powerslide_duration', 'avg_speed_percentage',
       'percent_slow_speed', 'percent_boost_speed', 'percent_supersonic_speed',
       'percent_ground', 'percent_low_air', 'percent_high_air',
       'avg_distance_to_ball', 'avg_distance_to_ball_possession',
       'avg_distance_to_ball_no_possession', 'avg_distance_to_mates',
       'time_defensiv

In [22]:
columns_to_convert = [
    'bpm', 'amount_collected', 'amount_stolen', 'amount_collected_big',
    'amount_stolen_big', 'amount_collected_small', 'amount_stolen_small',
    'amount_overfill', 'amount_overfill_stolen', 'amount_used_while_supersonic',
    'avg_speed', 'count_powerslide', 'avg_distance_to_ball',
    'avg_distance_to_ball_possession', 'avg_distance_to_ball_no_possession'
]

df_dropped1[columns_to_convert] = df_dropped1[columns_to_convert].astype('float64')


In [23]:
df_dropped1['win'] = df_dropped1['win'].astype(bool)

In [15]:
df_dropped1.dtypes

replay_id                              object
player_id                              object
player_name                            object
team_color                             object
bpm                                   float64
bcpm                                  float64
amount_collected                      float64
amount_stolen                         float64
amount_collected_big                  float64
amount_stolen_big                     float64
amount_collected_small                float64
amount_stolen_small                   float64
amount_overfill                       float64
amount_overfill_stolen                float64
amount_used_while_supersonic          float64
percent_zero_boost                    float64
percent_full_boost                    float64
percent_boost_0_25                    float64
percent_boost_25_50                   float64
percent_boost_50_75                   float64
percent_boost_75_100                  float64
avg_speed                         

In [28]:
df_dropped1.tail(6)

,replay_id,player_id,player_name,team_color,bpm,bcpm,amount_collected,amount_stolen,amount_collected_big,amount_stolen_big,amount_collected_small,amount_stolen_small,amount_overfill,amount_overfill_stolen,amount_used_while_supersonic,percent_zero_boost,percent_full_boost,percent_boost_0_25,percent_boost_25_50,percent_boost_50_75,percent_boost_75_100,avg_speed,time_supersonic_speed,time_boost_speed,time_slow_speed,time_ground,time_low_air,time_high_air,time_powerslide,count_powerslide,avg_powerslide_duration,avg_speed_percentage,percent_slow_speed,percent_boost_speed,percent_supersonic_speed,percent_ground,percent_low_air,percent_high_air,avg_distance_to_ball,avg_distance_to_ball_possession,avg_distance_to_ball_no_possession,avg_distance_to_mates,time_defensive_third,time_neutral_third,time_offensive_third,time_defensive_half,time_offensive_half,time_behind_ball,time_infront_ball,time_most_back,time_most_forward,time_closest_to_ball,time_farthest_from_ball,percent_defensive_third,percent_offensive_third,percent_neutral_third,percent_behind_ball,percent_infront_ball,percent_most_back,percent_most_forward,percent_closest_to_ball,percent_farthest_from_ball,win
89992,fffd08c7-c7e5-4d40-ad3e-c8ac947dfad8,8b504757dfb64360b9cb2c3ba9dfdde5,FanisDaGoat.,blue,468.0,478.24402,3290.0,631.0,2683.0,387.0,607.0,244.0,519.0,123.0,120.0,14.533869,11.229285,21.946232,23.785980,16.053812,38.213970,1602.0,87.22,169.50,169.57,177.82,207.81,40.66,4.60,47.0,0.10,69.652176,39.778084,39.761665,20.460249,41.713387,48.748505,9.538108,2370.0,2168.0,2596.0,3832.0,189.70,118.65,117.94,250.26,176.03,290.46,135.83,103.9,165.8,195.6,106.7,44.500220,27.666610,27.833164,68.136720,31.863287,25.172012,40.168620,47.388313,25.850372,True
89993,fffd08c7-c7e5-4d40-ad3e-c8ac947dfad8,525067367a89426aa4270ca8a6f3cddb,OG evdo,blue,355.0,371.40225,2555.0,576.0,1991.0,441.0,564.0,135.0,244.0,67.0,119.0,12.966373,17.298187,30.677387,17.984210,20.555101,30.783304,1468.0,50.79,166.49,210.24,235.23,180.23,12.06,8.54,75.0,0.11,63.826088,49.176643,38.943207,11.880145,55.021988,42.157093,2.820921,2792.0,2542.0,2967.0,3722.0,187.18,137.28,103.07,259.33,168.19,315.37,112.16,120.1,115.1,121.4,113.5,43.781723,24.108250,32.110027,73.765590,26.234417,29.096811,27.885454,29.411764,27.497818,True
89994,fffd08c7-c7e5-4d40-ad3e-c8ac947dfad8,11563443cc1d4d0f89890bd00f080323,Yeray231102,blue,249.0,280.98650,1933.0,672.0,1552.0,525.0,381.0,147.0,565.0,174.0,71.0,5.128888,27.921795,15.048867,17.765680,27.900530,39.284916,1334.0,20.25,139.27,267.03,250.36,149.19,26.99,6.21,54.0,0.11,58.000000,62.602276,32.650333,4.747392,58.695553,34.976790,6.327660,3696.0,3696.0,3804.0,4144.0,220.45,126.99,79.11,288.33,138.21,324.12,102.43,192.0,129.8,94.2,198.4,51.682102,18.546478,29.771423,75.986404,24.013597,46.516132,31.446844,22.821978,48.066673,True
89995,fffd08c7-c7e5-4d40-ad3e-c8ac947dfad8,76561198345606853,Tony,orange,360.0,359.04642,2470.0,503.0,1800.0,367.0,670.0,136.0,218.0,33.0,273.0,9.470394,15.590173,30.955467,18.796312,17.629795,32.618423,1518.0,42.68,187.89,194.40,259.88,149.08,16.00,10.63,108.0,0.10,66.000000,45.744408,44.212536,10.043062,61.153988,35.080948,3.765060,2911.0,2888.0,2961.0,3788.0,202.95,150.38,71.64,289.54,135.43,307.55,117.42,138.7,131.3,152.9,129.3,47.756313,16.857660,35.386030,72.369810,27.630188,33.603060,31.810251,37.043316,31.325708,False
89996,fffd08c7-c7e5-4d40-ad3e-c8ac947dfad8,76561198162589533,Toma,orange,376.0,397.42220,2734.0,1141.0,1905.0,895.0,829.0,246.0,216.0,123.0,207.0,9.991278,6.873243,30.487072,27.771498,18.544798,23.196632,1497.0,52.69,165.16,206.17,237.72,159.21,27.08,8.26,62.0,0.13,65.086960,48.622704,38.950993,12.426300,56.064716,37.548645,6.386642,3122.0,2949.0,3275.0,3970.0,205.19,118.40,100.42,262.78,161.23,277.79,146.22,127.3,172.0,139.5,155.1,48.392730,23.683405,27.923872,65.514960,34.485035,30.841166,41.670704,33.796880,37.576317,False
89997,fffd08c7-c7e5-4d40-ad3e-c8ac947dfad8,ef443d0c3b574d70ab8f19ce0055c192,Inzok,orange,287.0,339.71313,2337.0,17

### Note
Due to some errors occurring in the querying of the API, I ended up with two seperate dataframes that I had to combine into one before more processing occurred. Below, I am just loading in, checking the shape of, and concatenating them together to create my final dataset of approx. 300,000 rows for modeling.

In [ ]:
# df_dropped2 = pd.read_csv('C:/Users/nickh/Desktop/BrainStation Course/Capstone Project RL/FullData0to35000.csv')

In [30]:
df_dropped1.shape

(89998, 63)

In [31]:
df_dropped2.shape

(209994, 63)

In [29]:
df_dropped2 = df_dropped2.drop(columns=[
    'Unnamed: 0'])

In [32]:
# combining the data pulled from the API. df_dropped1 is games 35,000-50,000, df_dropped2 is games 1-35,000
df_combined = pd.concat([df_dropped2, df_dropped1], ignore_index=True)


In [33]:
df_combined.shape

(299992, 63)

### Filling Nulls
There were very few nulls in this dataset to begin with, so picking either median or mean to fill the values with was a fairly arbitrary choice, as the tiny number of filled nulls would hardly affect the outcome of the modeling. I decided to go with median, as the data on ballchasing.com tends to have more stats from higher level players (as higher level players are more likely to set up their account to upload their games to ballchasing so they can track their own stats), and so to mitigate a higher than average value for the nulls, median is less affected by outliers.

In [135]:
df_combined.describe()

,bpm,bcpm,amount_collected,amount_stolen,amount_collected_big,amount_stolen_big,amount_collected_small,amount_stolen_small,amount_overfill,amount_overfill_stolen,amount_used_while_supersonic,percent_zero_boost,percent_full_boost,percent_boost_0_25,percent_boost_25_50,percent_boost_50_75,percent_boost_75_100,avg_speed,time_supersonic_speed,time_boost_speed,time_slow_speed,time_ground,time_low_air,time_high_air,time_powerslide,count_powerslide,avg_powerslide_duration,avg_speed_percentage,percent_slow_speed,percent_boost_speed,percent_supersonic_speed,percent_ground,percent_low_air,percent_high_air,avg_distance_to_ball,avg_distance_to_ball_possession,avg_distance_to_ball_no_possession,avg_distance_to_mates,time_defensive_third,time_neutral_third,time_offensive_third,time_defensive_half,time_offensive_half,time_behind_ball,time_infront_ball,time_most_back,time_most_forward,time_closest_to_ball,time_farthest_from_ball,percent_defensive_third,percent_offensive_third,percent_neutral_third,percent_behind_ball,percent_infront_ball,percent_most_back,percent_most_forward,percent_closest_to_ball,percent_farthest_from_ball
count,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000,299958.000000
mean,356.721074,371.613572,2008.762210,436.299515,1400.192634,257.519119,608.569576,178.780396,324.787060,58.698804,232.710963,10.461253,13.231946,29.837539,21.905325,19.397195,28.832937,1493.792848,41.318924,131.511913,164.061345,197.879003,124.429632,14.583557,6.260221,53.621970,0.128237,64.947515,48.760191,38.978908,12.259902,58.746219,36.928640,4.324141,2943.342235,2842.824645,3029.835367,3668.355550,163.612576,103.345352,69.934237,218.364267,118.527515,240.822289,96.069881,110.793172,108.350531,108.307934,110.720358,48.661613,20.704053,30.633334,71.497575,28.501425,34.140379,33.374727,33.358551,34.117561
std,74.835752,70.824740,599.656231,222.729680,490.142342,178.921079,205.446954,83.581382,177.644404,59.945642,154.921836,4.966903,5.875364,8.943757,5.301588,5.232974,7.876713,110.389448,20.203092,34.847425,46.734676,50.081136,33.117038,8.127276,2.987807,28.686376,0.067063,4.799541,8.326729,5.147212,5.275552,6.162220,5.168731,2.220838,301.681145,386.518963,331.614549,270.542528,44.795918,27.641071,24.662545,55.164194,36.178529,58.421622,28.290713,33.387494,31.558895,29.955047,32.003610,7.715626,5.732207,4.317648,5.718813,5.715052,6.718174,6.059054,5.158925,6.027962
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,660.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.100000,0.100000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.209952,0.218914,0.218914,0.209952
25%,313.000000,326.527375,1660.000000,277.000000,1083.000000,112.000000,481.000000,121.000000,197.000000,10.000000,121.000000,6.903859,9.031131,23.796346,18.376126,15.852438,23.507699,1426.000000,26.760000,115.020000,137.140000,175.180000,107.580000,8.640000,4.270000,34.000000,0.100000,62.000000,43.045268,35.962467,8.406734,54.589261,33

In [37]:
columns_to_fill = [
    "time_most_back", "time_most_forward", "time_closest_to_ball", "time_farthest_from_ball",
    "percent_most_back", "percent_most_forward", "percent_closest_to_ball", "percent_farthest_from_ball", "avg_distance_to_mates"
]

for column in columns_to_fill:
    median_value = df_combined[column].median()
    df_combined[column] = df_combined[column].fillna(median_value)

In [79]:
# filling in nulls for a column that I missed above
median_value = df_combined['avg_distance_to_mates'].median()
df_combined['avg_distance_to_mates'] = df_combined['avg_distance_to_mates'].fillna(median_value)

In [ ]:
# saving the combined dataset to my computer

# df_combined.to_csv('C:/Users/nickh/Desktop/BrainStation Course/Capstone Project RL/FULLDATA0to50000DroppedAndReady.csv')

### Important EDA Finding
There were games that had less than 6 players (due to someone not loading into the game at the start, which happens periodically), so below I am first grouping by replay ID and checking if there were 5 or less rows for each unique replay ID to determine which IDs had a missing player, then since there were only about 10 instances of this in all the data, I hand copy-pasted the replay IDs into the second and third code cells below to double check and then remove those rows respectively.

In [134]:
incomplete_games = df_combined.groupby('replay_id').filter(lambda x: len(x) <= 5)

print(incomplete_games['replay_id'].unique())

In [69]:
# searching by ID for games that had less than 6 players to verify before deleting those rows

filtered_df = df_combined.loc[df_combined['replay_id'] == 'dce7a117-5a02-4600-86b3-8b86a55597b8']

filtered_df


,replay_id,player_id,player_name,team_color,bpm,bcpm,amount_collected,amount_stolen,amount_collected_big,amount_stolen_big,amount_collected_small,amount_stolen_small,amount_overfill,amount_overfill_stolen,amount_used_while_supersonic,percent_zero_boost,percent_full_boost,percent_boost_0_25,percent_boost_25_50,percent_boost_50_75,percent_boost_75_100,avg_speed,time_supersonic_speed,time_boost_speed,time_slow_speed,time_ground,time_low_air,time_high_air,time_powerslide,count_powerslide,avg_powerslide_duration,avg_speed_percentage,percent_slow_speed,percent_boost_speed,percent_supersonic_speed,percent_ground,percent_low_air,percent_high_air,avg_distance_to_ball,avg_distance_to_ball_possession,avg_distance_to_ball_no_possession,avg_distance_to_mates,time_defensive_third,time_neutral_third,time_offensive_third,time_defensive_half,time_offensive_half,time_behind_ball,time_infront_ball,time_most_back,time_most_forward,time_closest_to_ball,time_farthest_from_ball,percent_defensive_third,percent_offensive_third,percent_neutral_third,percent_behind_ball,percent_infront_ball,percent_most_back,percent_most_forward,percent_closest_to_ball,percent_farthest_from_ball,win
181339,dce7a117-5a02-4600-86b3-8b86a55597b8,Bert_The_Herp,Bert_The_Herp,blue,347.0,398.73416,21.0,0.0,0.0,0.0,21.0,0.0,0.0,0.0,0.0,0.000000,0.0,20.000000,80.000000,0.0,0.000000,1265.0,0.83,0.24,4.90,4.90,1.07,0.0,0.07,1.0,0.07,55.000000,82.077060,4.020101,13.902848,82.077050,17.922949,0.0,4455.0,0.0,0.0,3317.0,5.97,0.00,0.0,5.97,0.0,5.97,0.0,2.3,0.1,0.1,2.3,100.00000,0.0,0.000000,100.0,0.0,72.784805,3.164557,3.164557,72.784805,False
181340,dce7a117-5a02-4600-86b3-8b86a55597b8,027e59d7cf3f4d7d99ab7a37d13c18f5,Thiswasntaken,blue,152.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,4.967602,95.032394,0.0,0.000000,1295.0,0.00,1.24,4.67,4.57,1.34,0.0,0.00,0.0,0.00,56.304348,79.018616,20.981388,0.000000,77.326570,22.673433,0.0,2256.0,0.0,0.0,3317.0,4.97,0.94,0.0,5.90,0.0,5.90,0.0,110.5,5.9,5.9,110.7,84.09475,0.0,15.905246,100.0,0.0,33.929431,186.708860,186.708860,33.934000,False
181341,dce7a117-5a02-4600-86b3-8b86a55597b8,5f7b69f656674aacabe74721ba8fb4f7,DTsar,orange,370.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,17.962965,82.037040,0.0,0.000000,1382.0,0.13,1.20,4.57,5.67,0.23,0.0,0.00,0.0,0.00,60.086956,77.457630,20.338982,2.203390,96.101690,3.898305,0.0,2154.0,0.0,0.0,3088.0,4.97,0.94,0.0,5.90,0.0,5.90,0.0,110.5,2.2,2.2,110.7,84.09475,0.0,15.905246,100.0,0.0,33.929431,69.620255,69.620255,33.934000,True
181342,dce7a117-5a02-4600-86b3-8b86a55597b8,97845fa684db43af9d086c03b5aeecea,indimane,orange,57.0,208.86075,11.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,100.000000,0.0,0.000000,328.0,0.00,0.00,5.97,4.83,1.14,0.0,0.00,0.0,0.00,14.260870,100.000000,0.000000,0.000000,80.904526,19.095478,0.0,4315.0,0.0,0.0,2221.0,5.97,0.00,0.0,5.97,0.0,5.97,0.0,2.3,108.3,0.1,1.5,100.00000,0.0,0.000000,100.0,0.0,72.784805,33.203011,3.164557,47.468353,True
181343,dce7a117-5a02-4600-86b3-8b86a55597b8,cb2605e841a244adbb87f2347bbd6042,zxcursed,orange,329.0,1898.73410,100.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,32.911392,0.0,82.866560,0.000000,0.0,17.133442,1289.0,0.00,1.37,4.60,4.73,1.24,0.0,0.00,0.0,0.00,56.043480,77.051926,22.948074,0.000000,79.229480,20.770520,0.0,4046.0,0.0,0.0,2345.0,5.97,0.00,0.0,5.97,0.0,5.97,0.0,110.5,0.1,108.7,0.8,100.00000,0.0,0.000000,100.0,0.0,33.929431,3.164557,33.222007,25.316456,True


In [70]:
# removing games in the data that have less than 6 players
df_combined = df_combined[df_combined['replay_id'] != 'dce7a117-5a02-4600-86b3-8b86a55597b8']


In [85]:
# double checking the shape and nulls before proceeding to modeling
df_combined.shape

(299958, 63)

In [80]:
df_combined.isna().sum()

replay_id                             0
player_id                             0
player_name                           0
team_color                            0
bpm                                   0
bcpm                                  0
amount_collected                      0
amount_stolen                         0
amount_collected_big                  0
amount_stolen_big                     0
amount_collected_small                0
amount_stolen_small                   0
amount_overfill                       0
amount_overfill_stolen                0
amount_used_while_supersonic          0
percent_zero_boost                    0
percent_full_boost                    0
percent_boost_0_25                    0
percent_boost_25_50                   0
percent_boost_50_75                   0
percent_boost_75_100                  0
avg_speed                             0
time_supersonic_speed                 0
time_boost_speed                      0
time_slow_speed                       0


# Preparing the Data and Modeling
Directly below is my final list of features to include in the modeling.

Before modeling can happening, I need to extract my feature columns from the dataset and then flatten and append together the 3 rows that make up one team. I groupby replay ID and loop through team color to do this operation for each team before being able to feed the final data into the models.

I chose to test out 3 different models (Logistic Regression, Support Vector Machines and XGBoost), aftering attempting to get a Neural Network to work, with little to no success (on the NN). Below is a brief description of why I chose these 3 models:

- Logistic Regression because of it's efficiency and relative easy of interpretability in predicting binary outcomes
- SVM because of it's effectiveness in higher dimensional spaces and possibility to capture more complicated relationships
- XGBoost because it's good at not overfitting while still more efficient than SVM and still able to capture some of those non-linear relationships



In [136]:
feature_cols = ['bpm', 'bcpm',
       'amount_collected', 'amount_stolen', 'percent_zero_boost',
       'percent_full_boost', 'avg_speed', 'time_powerslide',
       'count_powerslide',
       'percent_slow_speed', 'percent_boost_speed', 'percent_supersonic_speed',
       'percent_ground', 'percent_low_air', 'percent_high_air', 'avg_distance_to_ball_possession',
       'avg_distance_to_ball_no_possession', 'avg_distance_to_mates',
       'percent_defensive_third', 'percent_offensive_third',
       'percent_neutral_third', 'percent_behind_ball', 'percent_infront_ball']

In [ ]:
def prepare_game_data(df):
    games = []
    for _, game in df.groupby('replay_id'):
        game_data = game.sort_values(by=['team_color', 'player_id'])
        features = game[feature_cols].values.flatten()
        win = game[game['team_color'] == 'blue']['win'].iloc[0]
        games.append(np.append(features, win))
    return pd.DataFrame(games)

## Logistic Regression
Feeding the data into this model resulted in extremely fast training and a final accuracy of 80-81% (depending on the random state and after more feature engineering (as I periodically tried these models throughout my feature engineering).

In [121]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix


df_games = prepare_game_data(df_combined)

X = df_games.iloc[:, :-1]
y = df_games.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [123]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=1000)

In [124]:
y_pred = model.predict(X_test_scaled)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.81      0.80      0.80      5011
         1.0       0.80      0.81      0.80      4988

    accuracy                           0.80      9999
   macro avg       0.80      0.80      0.80      9999
weighted avg       0.80      0.80      0.80      9999

[[4027  984]
 [ 971 4017]]


Below is how to test new data on the model

In [ ]:
new_game_features = df_new_game[feature_cols].values.flatten().reshape(1, -1)

new_game_features_scaled = scaler.transform(new_game_features)

new_game_prediction = model.predict(new_game_features_scaled)
new_game_prob = model.predict_proba(new_game_features_scaled)

print("predicted outcome:", "win" if new_game_prediction[0] == 1 else "loss")
print("win/loss prob:", new_game_prob)

## SVM
This model took extremely long to train and ended up with very similar results. I believe that with a bit more feature engineering and more data, this model could possibly outperform logistic regression. This is something I will continue to work on in the Sprint that involves advanced modeling.

In [129]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

svm_model = SVC(probability=True, kernel='rbf', random_state=37)

svm_model.fit(X_train_scaled, y_train)

svm_predictions = svm_model.predict(X_test_scaled)

print("SVM Classification Report:")
print(classification_report(y_test, svm_predictions))
print("SVM Confusion Matrix:")
print(confusion_matrix(y_test, svm_predictions))


SVM Classification Report:
              precision    recall  f1-score   support

         0.0       0.80      0.81      0.81      5011
         1.0       0.81      0.80      0.80      4988

    accuracy                           0.80      9999
   macro avg       0.80      0.80      0.80      9999
weighted avg       0.80      0.80      0.80      9999

SVM Confusion Matrix:
[[4051  960]
 [1001 3987]]


## XGBoost
This model performed the worst and took slightly longer than Logistic Regression to train. Potentially messing with parameters such as the evaluation metric could improve this model's effectiveness in future advanced modeling.

In [128]:
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix

xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=37)

xgb_model.fit(X_train_scaled, y_train)

xgb_predictions = xgb_model.predict(X_test_scaled)

# Evaluation
print("XGBoost Classification Report:")
print(classification_report(y_test, xgb_predictions))
print("XGBoost Confusion Matrix:")
print(confusion_matrix(y_test, xgb_predictions))


XGBoost Classification Report:
              precision    recall  f1-score   support

         0.0       0.80      0.79      0.79      5011
         1.0       0.79      0.80      0.79      4988

    accuracy                           0.79      9999
   macro avg       0.79      0.79      0.79      9999
weighted avg       0.79      0.79      0.79      9999

XGBoost Confusion Matrix:
[[3938 1073]
 [ 992 3996]]


## Modeling conclusion
There is a clear winner here in the logistic regression model, as it performed similarly to SVM, while taking a way shorter period of time to train. 

I will be looking into applying a Neural Network to this data in future modeling as I think there are some complicated relationships in the data that cannot be fully represented by the above models, but in my intial exploration of NN's I could not get it to be any better than a coin flip.

I will also look even more closely at feature engineering and tweaking of parameters to optimize these models, as well as search for any other models I may have overlooked that could possibly do an even better prediction job. 